<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/04_mcp/01_mcp_stdio_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q mcp uvicorn nest_asyncio

In [26]:
import nest_asyncio
import sys
import os

In [27]:
# 1. Apply the patch allows nested async loops in Colab
nest_asyncio.apply()

In [36]:
%%writefile server.py
import logging
import sys

# 1. Silence all logs (Fixes the "INFO Processing..." noise)
logging.basicConfig(level=logging.ERROR)

from mcp.server.fastmcp import FastMCP

mcp = FastMCP("MathServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    return a * b

if __name__ == "__main__":
    mcp.run()

Overwriting server.py


In [37]:
%%writefile client.py
import asyncio
import sys
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def run():
    # 1. Launch server in UNBUFFERED mode ("-u")
    server_params = StdioServerParameters(
        command=sys.executable,
        args=["-u", "server.py"], # <--- THE FIX
    )

    print("🔌 Client connecting...")

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Test Add
            print(">> Requesting: add(100, 50)")
            result = await session.call_tool("add", arguments={"a": 100, "b": 50})
            print(f"✅ Result: {result.content[0].text}")

            # Test Multiply
            print(">> Requesting: multiply(5, 5)")
            result = await session.call_tool("multiply", arguments={"a": 5, "b": 5})
            print(f"✅ Result: {result.content[0].text}")

if __name__ == "__main__":
    asyncio.run(run())

Overwriting client.py


In [38]:
!python client.py

🔌 Client connecting...
>> Requesting: add(100, 50)
✅ Result: 150
>> Requesting: multiply(5, 5)
✅ Result: 25
